## Document Searcher Class
## To do: Excel load multiple documents

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import cohere
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings
COHERE_API_KEY  = os.environ['COHERE_API_KEY']
GEMINI_API_KEY  = os.environ['GEMINI_API_KEY']
cohere_key = COHERE_API_KEY
co = cohere.Client(cohere_key)
gt = pd.read_excel('data/ground_truth.xlsx')

In [179]:
def letter_tokenizer(word,n = 3):
    tokens = []
    for i in range(len(word) - n + 1):
        tokens.append(word[i:i+n])
    return tokens

def preprocess_text(text):
    tokens = letter_tokenizer(text)
    return ' '.join(tokens)


def generate_prompt(q,retrieved_documents,group_key):
    if retrieved_documents == None:
        return "Not confident enough to generate prompt"
    else:
        context_string = ""
        for section,context in retrieved_documents.items():
            context_string += '-------' + group_key[section] + '-------'
            context_string += ' '.join(context)
            context_string += '\n'
    
        prompt = """ 
    Based on the context below answer the given question. Given Question: {}
    ----------------------------- Context -----------------------------
    {}
        """.format(q,context_string) 
    
        return prompt

def rerank_documents(q,context_group,frac = 0.7):
    if context_group == None:
        return None
    reranked_context = {}
    for group,context in context_group.items():
        n = round(frac*len(context))
        if n > 0:
            res = co.rerank(query = q, documents = context, top_n = n , model = 'rerank-multilingual-v3.0')
            reranked_list = []
            for doc in res.results:
              doc = context[doc.index]
              reranked_list.append(doc)
            reranked_context[group] = reranked_list
        else:
            reranked_context[group] = []
    return reranked_context

In [180]:
class DocSearcher:
    def __init__(self,docs,embedding_model):
        docs['context_embedding'] = docs['Question'].apply(lambda x: embedding_model.embed_query(x))
        docs['Group_short'] = docs['Group_short'].astype(str)
        group_key = docs[['Group_short','Group']].drop_duplicates()
        group_key = dict(zip(group_key['Group_short'], group_key['Group']))
        self.emb = embedding_model
        self.docs = docs
        self.group_key = group_key

    def _get_context_from_scores(self,section_scores,num_section = 5,k = 15):
        docs = self.docs
        group_key = self.group_key
        is_confident = True
        for section,score in section_scores.items():
            if score < 0:
                section_scores[section] = 0
        section_scores = sorted(section_scores.items(), key=lambda x: x[1],reverse = True)
        total_scores = sum(value for key, value in section_scores[0:num_section])
        if total_scores <= 0.25:
            is_confident = False
            context = None
            return is_confident, context
        else:
            context = {}
            for score in section_scores[0:num_section]:
                section = score[0]
                filtered_docs = docs[docs['Group_short']==section].reset_index(drop = True)
                n_documents = round(k*score[1]/total_scores)
                chosen_row = filtered_docs.iloc[0:n_documents]
                context[section] = chosen_row['q_a'].tolist()
            return is_confident,context     

    def query_documents(self,q,k = 15,method = 'all',num_section = 5):
        docs = self.docs
        docs['q_a'] = '\n Question: ' + docs['Question'] + '\n Answer: ' + docs['Answer']
        if method == 'all':
            scores_tfidf = self.get_section_tfidf_scores(q)
            scores_vector = self.get_section_vector_scores(q)
            scores_combined = {section: (scores_tfidf[section] + scores_vector[section])/2 for section in scores_vector}
            is_confident, context = self._get_context_from_scores(section_scores = scores_combined,num_section = num_section,k = k)
            if is_confident == True:
                return context
            else:
                return None

        elif method == 'tfidf':
            scores_tfidf = self.get_section_tfidf_scores(q)
            is_confident, context = self._get_context_from_scores(section_scores = scores_tfidf,num_section = num_section,k = k)
            if is_confident == True:
                return context
            else:
                return None
            
        elif method == 'vector':
            scores_vector = self.get_section_vector_scores(q)
            is_confident, context = self._get_context_from_scores(section_scores = scores_vector,num_section = num_section,k = k)
            if is_confident == True:
                return context
            else:
                return None

        elif method == 'fuzzy':
            pass
        else:
            raise NameError("Please input the correct method")

    def query_documents_with_prompt(self,q,k = 15,method = 'all'):
        context = self.query_documents(q,k,method)
        if context != None: 
            group_key = self.group_key
            context_string = ""
            for section,doc in context.items():
                context_string += '-------' + group_key[section] + '-------'
                context_string += ' '.join(doc)
                context_string += '\n'
            prompt = """
        Base the context below answer the given question. Given Question: {}
        ------------------------- Context -----------------------------
        {}
            """.format(q,context_string)
            return prompt
        else:
            return "Not confident enough to generate prompt"

    # Whole document scores
    def get_section_tfidf_scores(self,q):
        docs = self.docs
        docs_grouped = docs.groupby('Group_short')['Question'].apply(lambda x: ' '.join(x)).reset_index()
        docs_grouped['docs'] = docs_grouped['Group_short']+' '+docs_grouped['Question']
        docs_grouped['cleaned_document'] = docs_grouped['Question'].apply(preprocess_text)
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform(docs_grouped['cleaned_document'])
        cleaned_query = preprocess_text(q)
        query_vector = tfidf_vectorizer.transform([cleaned_query])
        similarities = cosine_similarity(query_vector, tfidf_matrix)
        relevant_docs_indices = similarities.argsort()[0][::-1]
        scores_tfidf = {}
        for i, idx in enumerate(relevant_docs_indices):
            scores_tfidf[docs_grouped.iloc[idx,:]['Group_short']] = similarities[0][idx]     
        return scores_tfidf

    def get_section_vector_scores(self,q):
        emb = self.emb
        docs = self.docs 
        query_embedding = np.array(emb.embed_query(q)).reshape(1,-1)
        context_embeddings = np.array(docs['context_embedding'].tolist())
        scores_vector = cosine_similarity(context_embeddings, query_embedding).flatten()
        docs['similarity_score_vector'] = scores_vector
        docs = docs.sort_values(by = 'similarity_score_vector',ascending = False) 
        self.docs = docs
        scores_vector = docs.groupby(['Group_short'])['similarity_score_vector'].mean().sort_values(ascending = False).to_dict()
        return scores_vector

In [181]:
d = DocSearcher(gt,embedding_model = HuggingFaceEmbeddings(model_name='mrp/simcse-model-m-bert-thai-cased'))

C:\Users\chana\anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [182]:
q = 'NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่'

In [183]:
d.get_section_vector_scores(q)

{'1.1': 0.32598045903847206,
 '1.7': 0.25978287263824473,
 '1.4': 0.1519963728969059,
 '1.3': 0.09605046657074767,
 '1.2': 0.09417312371266462,
 '1.6': 0.08272744734953184,
 '4.0': 0.04906059914176389,
 '3.0': 0.0368564587094577,
 '2.0': -0.002767239704413641,
 '1.5': -0.008370693207077416}

In [184]:
d.get_section_tfidf_scores(q)

{'1.1': 0.6549083363970302,
 '1.7': 0.3625609777308607,
 '1.6': 0.21056972351409237,
 '1.3': 0.2065193512150642,
 '2.0': 0.15484824084924378,
 '1.2': 0.14761062546312947,
 '1.4': 0.1362631440109982,
 '3.0': 0.13097331820955402,
 '1.5': 0.12898374244475638,
 '4.0': 0.06751230734678952}

In [185]:
retrieved_documents = d.query_documents(q)

In [186]:
retrieved_documents_tfidf = d.query_documents(q,method = 'tfidf')
retrieved_documents_tfidf

{'1.1': ['\n Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่\n Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี',
  '\n Question: การฝากหรือถอนเงินในบัญชีNRBA และ NRBS กรณี NR ขายหรือซื้อเงินตราต่างประเทศกับต่าง ธพ. จะต้องเรียกเอกสารหลักฐานอย่างไร\n Answer: ให้ ธพ. ที่ทำธุรกรรมกับ NR ยืนยันว่าได้ทำธุรกรรม ซื้อหรือขายเงินตราต่างประเทศกับ NR จริง',
  '\n Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่\n Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้',
  '\n Question: NR สามารถถอนเงินบาทจากบัญชี Resident Baht Account เพื่อซื้อเงินตราต่างประเทศ โดยไม่แสดง underlying (ใช้วงเงิน no underlying ตาม มาตรการป้องปรามการเก็งกำไรค่าเงินบาท) ได้หรือไม่\n Answer: ไม่สามารถทำได้ ธพ. จะต้องตรวจสอบว่าแหล่งที่มา ของเงินบาทของ NR ต้องไม่ได้มาจาก Negative list2 อย่างไร

In [187]:
retrieved_documents_vector = d.query_documents(q,method = 'vector')
retrieved_documents_vector

{'1.1': ['\n Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่\n Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี',
  '\n Question: การฝากหรือถอนเงินในบัญชีNRBA และ NRBS กรณี NR ขายหรือซื้อเงินตราต่างประเทศกับต่าง ธพ. จะต้องเรียกเอกสารหลักฐานอย่างไร\n Answer: ให้ ธพ. ที่ทำธุรกรรมกับ NR ยืนยันว่าได้ทำธุรกรรม ซื้อหรือขายเงินตราต่างประเทศกับ NR จริง',
  '\n Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่\n Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้',
  '\n Question: NR สามารถถอนเงินบาทจากบัญชี Resident Baht Account เพื่อซื้อเงินตราต่างประเทศ โดยไม่แสดง underlying (ใช้วงเงิน no underlying ตาม มาตรการป้องปรามการเก็งกำไรค่าเงินบาท) ได้หรือไม่\n Answer: ไม่สามารถทำได้ ธพ. จะต้องตรวจสอบว่าแหล่งที่มา ของเงินบาทของ NR ต้องไม่ได้มาจาก Negative list2 อย่างไร

In [188]:
print(generate_prompt(q,retrieved_documents,d.group_key))

 
    Based on the context below answer the given question. Given Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
    ----------------------------- Context -----------------------------
    -------1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ-------
 Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
 Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี 
 Question: การฝากหรือถอนเงินในบัญชีNRBA และ NRBS กรณี NR ขายหรือซื้อเงินตราต่างประเทศกับต่าง ธพ. จะต้องเรียกเอกสารหลักฐานอย่างไร
 Answer: ให้ ธพ. ที่ทำธุรกรรมกับ NR ยืนยันว่าได้ทำธุรกรรม ซื้อหรือขายเงินตราต่างประเทศกับ NR จริง 
 Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่
 Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้ 
 Question: NR สามารถถอนเงินบาทจากบัญชี Resident Baht Account เพ

In [189]:
print(d.query_documents_with_prompt(q))


        Base the context below answer the given question. Given Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
        ------------------------- Context -----------------------------
        -------1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ-------
 Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
 Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี 
 Question: การฝากหรือถอนเงินในบัญชีNRBA และ NRBS กรณี NR ขายหรือซื้อเงินตราต่างประเทศกับต่าง ธพ. จะต้องเรียกเอกสารหลักฐานอย่างไร
 Answer: ให้ ธพ. ที่ทำธุรกรรมกับ NR ยืนยันว่าได้ทำธุรกรรม ซื้อหรือขายเงินตราต่างประเทศกับ NR จริง 
 Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่
 Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้ 
 Question: NR สามารถถอนเงินบาทจากบัญชี Resident Baht Account

In [190]:
retrieved_documents

{'1.1': ['\n Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่\n Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี',
  '\n Question: การฝากหรือถอนเงินในบัญชีNRBA และ NRBS กรณี NR ขายหรือซื้อเงินตราต่างประเทศกับต่าง ธพ. จะต้องเรียกเอกสารหลักฐานอย่างไร\n Answer: ให้ ธพ. ที่ทำธุรกรรมกับ NR ยืนยันว่าได้ทำธุรกรรม ซื้อหรือขายเงินตราต่างประเทศกับ NR จริง',
  '\n Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่\n Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้',
  '\n Question: NR สามารถถอนเงินบาทจากบัญชี Resident Baht Account เพื่อซื้อเงินตราต่างประเทศ โดยไม่แสดง underlying (ใช้วงเงิน no underlying ตาม มาตรการป้องปรามการเก็งกำไรค่าเงินบาท) ได้หรือไม่\n Answer: ไม่สามารถทำได้ ธพ. จะต้องตรวจสอบว่าแหล่งที่มา ของเงินบาทของ NR ต้องไม่ได้มาจาก Negative list2 อย่างไร

In [191]:
reranked_context = rerank_documents(q,retrieved_documents)

In [192]:
print(generate_prompt(q,reranked_context,d.group_key))

 
    Based on the context below answer the given question. Given Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
    ----------------------------- Context -----------------------------
    -------1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ-------
 Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
 Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี 
 Question: NR สามารถโอนเงินเพื่อซื้อขายหลักทรัพย์และตรา สารทางการเงินอื่นจากบัญชี NRBA ไปเข้าบัญชี NRBS ได้หรือไม่
 Answer: หากมีเหตุผลชัดเจนสามารถยื่นขออนุญาตต่อ ธปท. เป็นรายกรณีได้ 
 Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่
 Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้ 
 Question: กรณี NR ลงทุนโดยตรงโดยทยอยซื้อหุ้นครั้งละไม่ถึง 10% ของหุ้นทั้งหมดของกิจการ ผ่านบัญชี NRBS เมื่อ

# Google's Gemini API

In [193]:
import google.generativeai as genai
from keys import GEMINI_API_KEY

In [194]:
p = generate_prompt(q,reranked_context,d.group_key)

In [195]:
print(p)

 
    Based on the context below answer the given question. Given Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
    ----------------------------- Context -----------------------------
    -------1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ-------
 Question: NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้หรือไม่
 Answer: ต้องขออนุญาต ธปท. เป็นรายกรณี 
 Question: NR สามารถโอนเงินเพื่อซื้อขายหลักทรัพย์และตรา สารทางการเงินอื่นจากบัญชี NRBA ไปเข้าบัญชี NRBS ได้หรือไม่
 Answer: หากมีเหตุผลชัดเจนสามารถยื่นขออนุญาตต่อ ธปท. เป็นรายกรณีได้ 
 Question: ในกรณีที่ NR ถอนเงินจากบัญชี NRBA หรือ NRBS ธพ. จะต้องเรียกเอกสารหลักฐานแสดงวัตถุประสงค์ การถอนหรือไม่
 Answer: ธพ. ไม่ต้องเรียกเอกสารแสดงวัตถุประสงค์การถอน เงินดังกล่าวจาก NR อย่างไรก็ตาม ธพ. ต้องดูแลให้ วัตถุประสงค์การถอนเงินเป็นไปตามวัตถุประสงค์ ของแต่ละประเภทบัญชี หรือหาก ธพ. ไม่แน่ใจ อาจเรียกเอกสารหลักฐานเพิ่มเติมได้ 
 Question: กรณี NR ลงทุนโดยตรงโดยทยอยซื้อหุ้นครั้งละไม่ถึง 10% ของหุ้นทั้งหมดของกิจการ ผ่านบัญชี NRBS เมื่อ

In [196]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro')
response = model.generate_content(p)
print(response.text)

I'm sorry, but I cannot answer this question. The document is structured as a series of FAQs, and it's difficult to extract the meaning of "NR", "NRBA", and "NRBS" from the provided text. Please provide a definition or more context for these terms. 



## Fuzzy Matching: Exploration

In [48]:
from fuzzywuzzy import fuzz

In [49]:
a = 'เกณ์การ KYB มีอะไรบ้าง'.lower()
b = "ผู้ประกอบธุรกิจ FX e-money สามารถ เติมเงินให้แก่ลูกค้าได้อย่างไรบ้าง".lower()
Ratio = fuzz.ratio(a,b)
print(Ratio)

29


In [50]:
query = 'NR ต้องการมาลงทุนในประเทศต้องทำอย่างไร?'.lower()
gt['fuzz'] = gt.apply(lambda x: fuzz.token_set_ratio(x['Question'],query), axis=1)

In [51]:
gt['Group'].unique()

array(['1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ',
       '1.2 การนำเงินตราต่างประเทศกลับเข้าประเทศ',
       '1.3 บัญชีเงินฝากเงินตราต่างประเทศ (FCD)',
       '1.4 เงินลงทุนโดยตรงหรือเงินให้กู้ในต่างประเทศ',
       '1.5 เงินลงทุนในหลักทรัพย์ต่างประเทศ',
       '1.6 อื่นๆ เช่น การทำธุรกรรมแทนกิจการในเครือ เงินกู้ การฝากเงินในบัญชีในต่างประเทศ',
       '1.7 กระบวนการ Know Your Business (KYB)',
       '2. การทำธุรกรรมอนุพันธ์', '3. ศูนย์บริหารเงิน', '4. FX E-money'],
      dtype=object)

In [52]:
gt.groupby(['Group_short'])['fuzz'].mean().sort_values(ascending = False)/gt.groupby(['Group_short'])['fuzz'].mean().sort_values(ascending = False).sum()

Group_short
1.7    0.119862
1.1    0.111084
1.4    0.105989
4.0    0.101966
1.3    0.097388
1.5    0.096679
1.2    0.095815
1.6    0.091937
3.0    0.090152
2.0    0.089128
Name: fuzz, dtype: float64

In [53]:
gt

,Question,Answer,Group,Group_short,context_embedding,similarity_score_vector,fuzz
0,กรณีบุคคลที่มีถิ่นที่อยู่นอกประเทศ (Non-reside...,- หากเป็นการลงทุนที่มีสัดส่วนตั้งแต่ 10% ของหุ...,1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ,1.1,"[0.057255879044532776, 0.0076107457280159, -0....",0.151052,47
1,กรณี NR ลงทุนโดยตรงโดยทยอยซื้อหุ้นครั้งละไม่ถึ...,- เมื่อ NR จะนำเงินค่าขายหุ้นฝากเข้าบัญชี หาก ...,1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ,1.1,"[-0.23912641406059265, 0.6383059024810791, 0.7...",0.447697,28
2,NR จะเปลี่ยนประเภทบัญชีจาก NRBA เป็น NRBS ได้ห...,ต้องขออนุญาต ธปท. เป็นรายกรณี,1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ,1.1,"[0.09722046554088593, 0.23035705089569092, 0.8...",0.308751,39
3,NR สามารถโอนเงินเพื่อซื้อขายหลักทรัพย์และตรา ส...,หากมีเหตุผลชัดเจนสามารถยื่นขออนุญาตต่อ ธปท. เป...,1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ,1.1,"[0.251921147108078, -0.34538087248802185, 0.09...",0.230281,32
4,กรณีโอนเงินเข้าผิดบัญชี จะต้องทำอย่างไร,เป็นกรณีที่ต้องขออนุญาตต่อ ธปท.,1.1 บัญชีเงินบาทของบุคคลที่มีถิ่นที่อยู่นอกประเทศ,1.1,"[-0.8113080859184265, 0.04314675182104111, -0....",0.033619,53
...,...,...,...,...,...,...,...
137,ผู้ประกอบธุรกิจ FX e-money สามารถ เติมเงินให้แ...,ผู้ประกอบธุรกิจสามารถเติมเงินให้ลูกค้าได้โดย 1...,4. FX E-money,4.0,"[-0.49097779393196106, 0.5335206985473633, 0.2...",-0.142624,42
138,ผู้ประกอบธุรกิจเงินอิเล็กทรอนิกส์สกุลเงินตรา ต...,ผู้ประกอบธุรกิจจะต้องดำรง float ในส่วนของ เงิน...,4. FX E-money,4.0,"[0.0936984121799469, 0.4437811076641083, -0.56...",-0.041669,28
139,ผู้ประกอบธุรกิจจ าเป็นต้องมีทั้งบัญชีเงินฝาก เ...,ผู้ประกอบธุรกิจต้องมีบัญชีเงินฝากเพื่อการด ารง...,4. FX E-money,4.0,"[0.24193742871284485, 0.9761651754379272, 0.60...",-0.041665,23
140,ผู้ประกอบธุรกิจ FX e-money สามารถซื้อ เงินตราต...,สามารถทำได้เพื่อวัตถุประสงค์ที่เกี่ยวข้องกับ ก...,4. FX E-money,4.0,"[-0.6967499852180481, 0.3803589642047882, 0.21...",-0.093363,31
